In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.io import loadmat
%matplotlib inline

In [15]:
train_X = pd.read_csv('../datasets/train_properties.csv')
train_y = pd.read_csv('../datasets/train_2016.csv')
train_y_wide = pd.read_csv('../datasets/train_2016_wide.csv')

In [16]:
train_X.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,2.0,NaN,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13
1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,1.0,NaN,143809.0,239679.0,2015.0,95870.0,2581.30,NaN,NaN,6.111002e+13
2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,1.0,NaN,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13
3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,...,1.0,NaN,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13
4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,...,2.0,NaN,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13


In [17]:
train_y.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [18]:
train_y_wide.head()

,parcelid,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12
0,10711738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0276,NaN,NaN,NaN,NaN
1,10711755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0182,NaN,NaN,NaN,NaN
2,10711805,NaN,NaN,NaN,NaN,-0.1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10711816,NaN,NaN,NaN,-0.0121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10711858,NaN,NaN,NaN,NaN,NaN,NaN,-0.0481,NaN,NaN,NaN,NaN,NaN


In [32]:
not_nas = (12 - train_y_wide.isnull().sum(axis=1))

In [36]:
not_nas.head()

0    1
1    1
2    1
3    1
4    1
dtype: int64

In [45]:
not_nas.value_counts()

1    90557
2      124
3        1
dtype: int64

In [115]:
train_y_ave = pd.concat([train_y_wide.iloc[:,0],train_y_wide.iloc[:,1:12].mean(axis = 1)], axis = 1)

In [116]:
train_y_ave.head()

,parcelid,0
0,10711738,0.0276
1,10711755,-0.0182
2,10711805,-0.1009
3,10711816,-0.0121
4,10711858,-0.0481


In [117]:
train_y_ave.shape

(90682, 2)

In [124]:
train = train_X.join(train_y_ave, on = 'parcelid', how = 'left', lsuffix = '_X', rsuffix = "_y")

In [127]:
train.shape, train_X.shape

((90682, 60), (90682, 58))

In [128]:
train.head()

,parcelid_X,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,parcelid_y,0
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13,NaN,NaN
1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,143809.0,239679.0,2015.0,95870.0,2581.30,NaN,NaN,6.111002e+13,NaN,NaN
2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13,NaN,NaN
3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,...,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13,NaN,NaN
4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,...,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13,NaN,NaN


In [136]:
train.shape

(90682, 60)

In [87]:
train_X.shape

(90682, 58)

In [89]:
pd.DataFrame(train_y_ave)

,0
0,0.0276
1,-0.0182
2,-0.1009
3,-0.0121
4,-0.0481
5,0.2897
6,0.0040
7,-0.0121
8,0.0139
9,-0.0450


In [129]:
train.iloc[:,58]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
         ..
90652   NaN
90653   NaN
90654   NaN
90655   NaN
90656   NaN
90657   NaN
90658   NaN
90659   NaN
90660   NaN
90661   NaN
90662   NaN
90663   NaN
90664   NaN
90665   NaN
90666   NaN
90667   NaN
90668   NaN
90669   NaN
90670   NaN
90671   NaN
90672   NaN
90673   NaN
90674   NaN
90675   NaN
90676   NaN
90677   NaN
90678   NaN
90679   NaN
90680   NaN
90681   NaN
Name: parcelid_y, dtype: float64